### TP2 (Q1), and Atelier MRR

#### Vithor Bertalan, Matricule 2135362

In [1]:
import pandas as pd
import scipy.io
import igraph as ig
import numpy as np
from statistics import mean
import time

## Loads matrix and CSVs
m = scipy.io.mmread('tp-matrix.dgt')
articles = pd.read_csv("12-articles.csv", sep=",")
matrix_names = pd.read_csv("tp-matrix-names.csv", sep=" ")

## Transpose the matrix for the PageRank calculation
mat = m.transpose()

## Calculates adjacencies - takes a VERY LONG time
#g = ig.Graph.Adjacency(mat)
## If this is the first execution, saves the graph to a local file (uncomment line below, and comment last line of this cell)
#g.write_graphml("file.gra")
## If the file is already written locally, reads it from this file, to speed executions
g = ig.Graph.Read_GraphML("file.gra", index=0)

### Q1, Part 1 - Calculate 10 recommendations of each of the articles in the 12-articles subset

In [3]:
## Calculates PageRank for the graph, using iGraph's standard method
pr = g.pagerank()

## Gets the indexes of the 12-articles set in the overall matrix
def gets_indexes():
    pos = []
    for i in articles["id"]:
        count = 0
        for j in matrix_names["x"]:
            if (i==j):
                pos.append(count)
            count+=1
    return pos

## Classifies the recommendations of a given index, based on their PageRanks
def gets_recommendations(i):
    rows, cols = m.tocsr()[i].nonzero()
    ranks = np.empty([0, 2])
    for j in cols:
        ranks = np.vstack([ranks,[j,pr[j]]])
    return ranks[ranks[:, 1].argsort()[::-1]]

## Gets all the recommendations of the 12 articles set
def gets_articles_recommendations():
    pos = gets_indexes()
    count = 1
    for i in pos:
        recs = gets_recommendations(i)
        ## Adds one variable i, because index 0 is equivalent to line 1 in the dataset
        print("For paper number {}, number {} in the 12-articles set, the top-10 recommendations by PageRank are:".format(i+1,count))
        vector = recs[:,0][:10].astype(int)
        ## Adds one to the recommendations, because index 0 is equivalent to line 1 in the dataset
        print([x+1 for x in vector])
        count+=1      
        
gets_articles_recommendations()

For paper number 15089, number 1 in the 12-articles set, the top-10 recommendations by PageRank are:
[22252, 3269, 3277, 16362, 18172, 8132, 9137, 13231, 6568, 6106]
For paper number 35353, number 2 in the 12-articles set, the top-10 recommendations by PageRank are:
[338, 12764, 282, 5907, 12769, 25724, 2004, 12467, 333, 3187]
For paper number 50496, number 3 in the 12-articles set, the top-10 recommendations by PageRank are:
[21335, 30836, 13710]
For paper number 50497, number 4 in the 12-articles set, the top-10 recommendations by PageRank are:
[97, 107, 281]
For paper number 11636, number 5 in the 12-articles set, the top-10 recommendations by PageRank are:
[1642, 6095, 1368, 4941, 33, 2859, 2083, 1323, 2271, 6204]
For paper number 12593, number 6 in the 12-articles set, the top-10 recommendations by PageRank are:
[17, 373, 2040, 9665, 3043, 4259, 8312, 3221, 372, 5197]
For paper number 36565, number 7 in the 12-articles set, the top-10 recommendations by PageRank are:
[8355, 696, 8

### Q1, Part 2 - Calculate the MRR for the references of each of the 12 articles.

In [4]:
## Creates ordered set of pageranks - the best pagerank will be first
def order_ranks():
    ranks = np.empty([0, 2])
    for i in range(len(pr)):
        ranks = np.vstack([ranks,[i,pr[i]]])
    return ranks[ranks[:, 1].argsort()[::-1]]

## Given the ordered set, finds the rank of a given index
def find_rank_index(i, ranks):
    count = 1
    for j in ranks[:,0]:
        if (i==j):
            return count
        count+=1

## Calculates mean reciprocal rank for each of the connections of a given index
def calculate_mrr(i):
    ## Orders the ranks
    ranks = order_ranks()
    rows, cols = m.tocsr()[i].nonzero()
    rrs = []
    
    ## For each of the references...
    for j in cols:
        ## Gets the rank of that index in the ordered list...
        index = find_rank_index(j,ranks)
        ## Calculates the reciprocal rank (1/rank)
        rrs.append(1/index)
    ## And returns the mean of those values
    return(mean(rrs))

## Calculates MRR for each of the 12 articles
def calculate_mrr_articles():
    pos = gets_indexes()
    count = 1
    for i in pos:
        mrr = calculate_mrr(i)
        ## Adds one to variable i, because index 0 is equivalent to line 1 in the dataset
        print("For paper number {}, number {} in the 12-articles set, the MRR is {}".format(i+1,count,mrr))
        count+=1

start_time = time.time()
calculate_mrr_articles()
print("The MRR for the 12 papers was calculated in {} seconds".format((time.time() - start_time)))

For paper number 15089, number 1 in the 12-articles set, the MRR is 3.4244840276777825e-05
For paper number 35353, number 2 in the 12-articles set, the MRR is 7.448341720585364e-05
For paper number 50496, number 3 in the 12-articles set, the MRR is 2.7973042562303706e-05
For paper number 50497, number 4 in the 12-articles set, the MRR is 6.425012369398826e-05
For paper number 11636, number 5 in the 12-articles set, the MRR is 4.283362564269419e-05
For paper number 12593, number 6 in the 12-articles set, the MRR is 0.00023706096911343852
For paper number 36565, number 7 in the 12-articles set, the MRR is 0.00011203084005298705
For paper number 12215, number 8 in the 12-articles set, the MRR is 2.592960811804186e-05
For paper number 18645, number 9 in the 12-articles set, the MRR is 0.0001025079188714051
For paper number 1594, number 10 in the 12-articles set, the MRR is 3.4371001611510045e-05
For paper number 35304, number 11 in the 12-articles set, the MRR is 0.00014033574625291597
For

### Atelier MRR - Calculate an alternative to MRR. In this case, we will use Mean Average Precision (MAP) and a custom metric.

In [5]:
## Calculates the average precision for the connections of a given index
## Source of idea: https://stats.stackexchange.com/questions/127041/mean-average-precision-vs-mean-reciprocal-rank
## Second source: http://sdsawtelle.github.io/blog/output/mean-average-precision-MAP-for-recommender-systems.html

def calculate_map(i):
    ## First, we order the papers paged on their PageRanks (the first has the higher PageRank)
    ranks = order_ranks()
    ## Then, we ge ordered list of recommendations (see method above)
    recs = gets_recommendations(i)
    aps = []
    precision_count = 1
    
    ## For each of the references...
    for j in recs[:,0]:
        ## Gets the rank of that index in the ordered list...
        index = find_rank_index(j,ranks)
        ## Calculates the precision (number of recommendations already given divided by index of current recommendation)
        aps.append(precision_count/index)        
        precision_count+=1

    ## Returns the multiplication of (1 divided by the number of recommendations) * sum of average precisions
    return(1/len(recs) * sum(aps))

## Calculates MAP for each of the 12 articles
def calculate_map_articles():
    pos = gets_indexes()
    count = 1
    for i in pos:
        map_ = calculate_map(i)
        ## Adds one to variable i, because index 0 is equivalent to line 1 in the dataset
        print("For paper number {}, number {} in the 12-articles set, the MAP is {}".format(i+1,count,map_))
        count+=1
        
start_time = time.time()
calculate_map_articles()
print("The MAP for the 12 papers was calculated in {} seconds".format((time.time() - start_time)))

For paper number 15089, number 1 in the 12-articles set, the MAP is 0.00018632370708764005
For paper number 35353, number 2 in the 12-articles set, the MAP is 0.00041236465494748456
For paper number 50496, number 3 in the 12-articles set, the MAP is 5.2378925833792124e-05
For paper number 50497, number 4 in the 12-articles set, the MAP is 9.641444363561471e-05
For paper number 11636, number 5 in the 12-articles set, the MAP is 0.00025412746894945906
For paper number 12593, number 6 in the 12-articles set, the MAP is 0.001303818573485607
For paper number 36565, number 7 in the 12-articles set, the MAP is 0.00032490886268498134
For paper number 12215, number 8 in the 12-articles set, the MAP is 0.0001859600374129505
For paper number 18645, number 9 in the 12-articles set, the MAP is 0.0007830707476872126
For paper number 1594, number 10 in the 12-articles set, the MAP is 0.0001891485895923592
For paper number 35304, number 11 in the 12-articles set, the MAP is 0.0004981745192381682
For p

### As we can see, MAP has similar results to MRR (e.g., article number 12593, number 6 in the articles set, is the best in both metrics), but the calculation time takes half the time of MRR - 12 seconds against 22 seconds. 

### In my point of view, if article 12593 is the top article, this means it has more influent citations than the other papers. 

### Let's confirm this information, by taking the first two reference in the ordered recommendation set of paper 12593, which are papers 17 and 373, as we can see in the beginning of the notebook.  

In [8]:
#For paper number 12593, number 6 in the 12-articles set, the top-10 recommendations by PageRank are:
#[17, 373, 2040, 9665, 3043, 4259, 8312, 3221, 372, 5197]

## Creates ordered set of pageranks - the best pagerank will be first
def order_ranks():
    ranks = np.empty([0, 2])
    for i in range(len(pr)):
        ranks = np.vstack([ranks,[i,pr[i]]])
    return ranks[ranks[:, 1].argsort()[::-1]]

## Given the ordered set, finds the rank of a given index
def find_rank_index(i, ranks):
    count = 1
    for j in ranks[:,0]:
        if (i==j):
            return count
        count+=1
        
ranks = order_ranks()
print(find_rank_index(16,ranks))
print(find_rank_index(372,ranks))

300
1031


### As we can see, paper 12593 is cited by, among others, the 300th and 1031th top papers by PageRank, making it very influential. Let's compare those results with the top 2 references of paper 50496, number 3 in the 12-articles set, the worst paper by MAP metrics:

In [32]:
#For paper number 50496, number 3 in the 12-articles set, the top-10 recommendations by PageRank are:
#[21335, 30836, 13710]

print(find_rank_index(21334,ranks))
print(find_rank_index(30835,ranks))

31666
31785


### Besides being cited by only 3 papers, the top-2 references of 50496 are quite down below the ordered set of recommendations: it is cited by the 31666th and 31785th papers. Therefore, we can see the MAP is a very good and fast metric to order the most important papers of our query.

### Now, let's create a custom metric: we are going to multiply 1/index by the pagerank of the recommendation, for each recommendation, and sum the result. This is done to penalize papers with few recommendations, even if it has recommendations with high pageranks. 


### The formula for our new metric is: metric = Σ 1/index(n) * PageRank(n), with Σ going from 1 to n, with n being the full subset of articles that cite that paper.

In [7]:
## Calculates the custom metric for a given subset

## Orders ranks by PageRank
def order_ranks():
    ranks = np.empty([0, 2])
    for i in range(len(pr)):
        ranks = np.vstack([ranks,[i,pr[i]]])
    return ranks[ranks[:, 1].argsort()[::-1]]

## Given the ordered set, finds the rank of a given index
def find_rank_index(i, ranks):
    count = 1
    for j in ranks[:,0]:
        if (i==j):
            return count
        count+=1

## Calculates new metric for each of the connections of a given index
def calculate_new_metric(i):
    ## Orders the ranks
    ranks = order_ranks()
    rows, cols = m.tocsr()[i].nonzero()
    rrs = []
    
    ## For each of the references...
    for j in cols:
        ## Gets the rank of that index in the ordered list...
        index = find_rank_index(j,ranks)
        ## Calculates the new metric rank (1/rank)
        rrs.append(1/index * pr(j))
    ## And returns the sum of those values multiplied by 
    return(sum(rrs))

## Calculates new metric for each of the 12 articles
def calculate_new_metric_articles():
    pos = gets_indexes()
    count = 1
    for i in pos:
        mrr = calculate_new_metric(i)
        ## Adds one to variable i, because index 0 is equivalent to line 1 in the dataset
        print("For paper number {}, number {} in the 12-articles set, the new metric is {}".format(i+1,count,mrr))
        count+=1

start_time = time.time()
calculate_mrr_articles()
print("The metric for the 12 papers was calculated in {} seconds".format((time.time() - start_time)))

For paper number 15089, number 1 in the 12-articles set, the MRR is 3.4244840276777825e-05
For paper number 35353, number 2 in the 12-articles set, the MRR is 7.448341720585364e-05
For paper number 50496, number 3 in the 12-articles set, the MRR is 2.7973042562303706e-05
For paper number 50497, number 4 in the 12-articles set, the MRR is 6.425012369398826e-05
For paper number 11636, number 5 in the 12-articles set, the MRR is 4.283362564269419e-05
For paper number 12593, number 6 in the 12-articles set, the MRR is 0.00023706096911343852
For paper number 36565, number 7 in the 12-articles set, the MRR is 0.00011203084005298705
For paper number 12215, number 8 in the 12-articles set, the MRR is 2.592960811804186e-05
For paper number 18645, number 9 in the 12-articles set, the MRR is 0.0001025079188714051
For paper number 1594, number 10 in the 12-articles set, the MRR is 3.4371001611510045e-05
For paper number 35304, number 11 in the 12-articles set, the MRR is 0.00014033574625291597
For

### Using this new metric, a paper with just one high-ranked PageRank citation would not be necessarily well placed, being surpassed by a paper with two not as well high-ranked PageRank citations. In this way, we penalize papers with not many recommendations. 

### Q3 - Get 10 recommendations by cosine similarity

In [51]:
from sklearn.metrics.pairwise import cosine_similarity

'index = the index of the paper to calculate the similarities'
'matrix = the adjacency matrix'
'n = the number of top similarities to return'
def get_similarities(index, matrix, n):
    
    ## Calculates cosine similarity matrix and the values of the given index
    cos_mat = cosine_similarity(matrix, dense_output=False)
    idx = cos_mat[index]
    
    ## Gets the index's values as a COO matrix
    idx = idx.tocoo()    
    ranks = np.empty([0, 2])
    
    ## Stacks the values in a numpy array to order and get the N highest values
    for _,row,sim in zip(idx.row, idx.col, idx.data):
        ranks = np.vstack([ranks,[row,sim]])    
    ranks = ranks[ranks[:, 1].argsort()[::-1]]
    
    ## Gets the n+1 values, since the first value will always be the index itself (1.0 cosine similarity)
    vector = ranks[:,0][:n+1].astype(int)
    
    ## For the elements with no similarities calculated (e.g., index 15089), returns null
    if (len(vector) == 0):
        print("No cosine similarities detected.")
        return
    
    ## Removes the first value
    vector = np.delete(vector, 0)

    print(vector)
    #print([x for x in vector])    
    
get_similarities(15087,m,10)

[20926 22705  3610  6352 20409 18939  9719  8809 11045 31747]


### Q2 - Creates thematic matrices

In [22]:
'Gets a matrix for each of the relations, sums the results, and converts it back to an adjacency matrix'
def create_thematic_matrix():
    
    ## Relation (1)
    rel1 = m

    ## Relation (-1)
    rel2 = m.transpose()

    ## Relation (1,1), and converts all values greater than 1 to 1
    rel3 = m * m
    rel3[rel3 > 1] = 1

    ## Relation (1,-1), and converts all values greater than 1 to 1
    rel4 = m * m.transpose()
    rel4[rel4 > 1] = 1

    ## Relation (-1, 1), and converts all values greater than 1 to 1
    rel5 = m.transpose() * m
    rel5[rel5 > 1] = 1

    ## Relation (1,-1,1), and converts all values greater than 1 to 1
    rel6 = m * m.transpose() * m
    rel6[rel6 > 1] = 1

    ## Relation (-1,1,1), and converts all values greater than 1 to 1
    rel7 = m.transpose() * m * m
    rel7[rel7 > 1] = 1
    
    ## Sums all values to a single matrix, and converts all values greater than 1 to 1
    them_mat = rel1 + rel2 + rel3 + rel4 + rel5 + rel6 + rel7
    them_mat[them_mat > 1] = 1
    
    return (them_mat)

In [12]:
scipy.sparse.save_npz('thematic_matrix.npz', them_mat)
##them_mat = scipy.sparse.load_npz('thematic_matrix.npz')

In [32]:
'Creates a matrix considering only the relations towards a given index - all other values will be zeroes'
'm = thematic matrix created above'
'index = the index of the paper'

def create_article_matrix(m, index):
    rows, cols = m.nonzero()
    for row,col in zip(rows,cols):
        if (row != index and col != index):
            m[row,col] = 0
    return (m)

def using_tocoo(m, index):
    coo_mat = m.tocoo()    
    for row,col,data in zip(coo_mat.row, coo_mat.col, coo_mat.data):
        if (row != index and col != index):
            m[row,col] = 0
    return (m) 

def using_tocoo_izip(m, index):
    coo_mat = m.tocoo()   
    for row,col,_ in itertools.izip(coo_mat.row, coo_mat.col, coo_mat.data):
        if (row != index and col != index):
            m[row,col] = 0
    return (m) 
        
#mat = create_article_matrix(them_mat,15088)
#mat = using_tocoo(them_mat,15088)
mat = using_tocoo_izip(them_mat,15088)

MemoryError: Unable to allocate 2.27 GiB for an array with shape (304576816,) and data type int64